# 8.1. F1-measure

На [лекции](https://colab.research.google.com/drive/12xpYdqi1S4y68FYHym2rZ3wzHSm8cSLm?usp=sharing) мы подробно обсудили, что доля правильных ответов - не самая лучшая метрика оценки качества классификации. Довольно часто доля правильных ответов даёт смещенную оценку качества и способна ввести в заблуждение. По этой причине мы пользуемся метриками качества, которые называются *точностью* и *полнотой*, а также их комбинацией - *F1-мерой*. Ваша задача - реализовать функции `precision`, `recall` и `f1`. На вход всех этих функий подаются два вектора: `y_true`: вектор правильных ответов и `y_pred`: вектор предсказаний.

В рамках выполнения этого задания можно использовать только модуль `numpy`.

In [7]:
import numpy as np

def precision(y_true, y_pred):
    """
    Вычисление точности (precision) для бинарной классификации.
    
    Args:
        y_true: Вектор истинных меток (0 или 1).
        y_pred: Вектор предсказанных меток (0 или 1).
    
    Returns:
        Значение precision.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    predicted_positives = np.sum(y_pred == 1)
    
    if predicted_positives == 0:
        return 0.0
    
    return true_positives / predicted_positives

def recall(y_true, y_pred):
    """
    Вычисление полноты (recall) для бинарной классификации.
    
    Args:
        y_true: Вектор истинных меток (0 или 1).
        y_pred: Вектор предсказанных меток (0 или 1).
    
    Returns:
        Значение recall.
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    true_positives = np.sum((y_pred == 1) & (y_true == 1))
    actual_positives = np.sum(y_true == 1)
    
    if actual_positives == 0:
        return 0.0
    
    return true_positives / actual_positives

def f1(y_true, y_pred):
    """
    Вычисление F1-меры для бинарной классификации.
    
    Args:
        y_true: Вектор истинных меток (0 или 1).
        y_pred: Вектор предсказанных меток (0 или 1).
    
    Returns:
        Значение F1-меры.
    """
    p = precision(y_true, y_pred)
    r = recall(y_true, y_pred)
    
    if p + r == 0:
        return 0.0
    
    return 2 * (p * r) / (p + r)

## Примечания

1. Проверить себя Вы можете, сравнив свои ответы с функциями `sklearn.metrics.precision_score`, `sklearn.metrics.recall_score` и `sklearn.metrics.f1_score`.

2. В реализуемых функциях запрещается использовать вывод любой информации на экран (в частности, недопустимо использование print()).

3. Для улучшения качества кода мы не рекомендуем заново реализовывать вычисление precision и recall внутри функции f1. Достаточно просто внутри функции f1 вызвать уже реализованные вами ранее функции self.precision() и self.recall().

4. Мы считаем, что в качестве меток классов используется 0 или 1 (случай меток -1 и 1 можно не рассматривать, но для более корректного решения рекомендуем также учесть и эту ситуацию).

5. Реализуемые функции не должны ничего выводить на экран - они должны возвращать значения соответствующих метрик.

# 8.2. Площади

В этой задаче Вам требуется сравнить значения метрик для 4 различных алгоритмов. Для этого отредактируйте следующий код так, чтобы он соответствовал сформулированному заданию.

In [1]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc


data = fetch_openml(data_id=42608)
X, y = data['data'].drop(columns='Outcome').values, data['data']['Outcome'].astype(int).values

X_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
x_test = scaler.transform(x_test)


tree = DecisionTreeClassifier(random_state=42)
lr = LogisticRegression(random_state=42)
knn = KNeighborsClassifier(n_neighbors=5)
svm = SVC(probability=True, random_state=42)


tree.fit(X_train, y_train)
lr.fit(X_train, y_train)
knn.fit(X_train, y_train)
svm.fit(X_train, y_train)


tree_roc_auc = roc_auc_score(y_test, tree.predict_proba(x_test)[:, 1])
lr_roc_auc = roc_auc_score(y_test, lr.predict_proba(x_test)[:, 1])
knn_roc_auc = roc_auc_score(y_test, knn.predict_proba(x_test)[:, 1])
svm_roc_auc = roc_auc_score(y_test, svm.predict_proba(x_test)[:, 1])


tree_precision, tree_recall, _ = precision_recall_curve(y_test, tree.predict_proba(x_test)[:, 1])
tree_pr_auc = auc(tree_recall, tree_precision)

lr_precision, lr_recall, _ = precision_recall_curve(y_test, lr.predict_proba(x_test)[:, 1])
lr_pr_auc = auc(lr_recall, lr_precision)

knn_precision, knn_recall, _ = precision_recall_curve(y_test, knn.predict_proba(x_test)[:, 1])
knn_pr_auc = auc(knn_recall, knn_precision)

svm_precision, svm_recall, _ = precision_recall_curve(y_test, svm.predict_proba(x_test)[:, 1])
svm_pr_auc = auc(svm_recall, svm_precision)


roc_aucs = {1: tree_roc_auc, 2: lr_roc_auc, 3: knn_roc_auc, 4: svm_roc_auc}
pr_aucs = {1: tree_pr_auc, 2: lr_pr_auc, 3: knn_pr_auc, 4: svm_pr_auc}

best_roc_algorithm = max(roc_aucs, key=roc_aucs.get)
best_pr_algorithm = max(pr_aucs, key=pr_aucs.get)

print(f"{best_roc_algorithm}{best_pr_algorithm}")


21


## Примечания

1. Обратите внимание, что StandardScaler может влиять на результаты работы алгоритмов. Поэтому рекомендуем его использовать.

2. Когда мы разбиваем данные на train и test, мы должны понимать, что **все** алгоритмы должны обучаться **только на train**. Test при обучении **не используется**. Более того, в реальной жизни мы вообще ничего не знаем про test. Поэтому StandardScaler нужно обучать только на X_train, а к X_test делать только transform (**не** fit_transform).